# Establish Pisces baseline by loading data, making triplots

In [ ]:

import numpy as np
from analyses.NHRC.nhrc_utils.analysis import DEFAULT_EVALUATION_DIR, SCENARIOS, STATIONARY_LOWER, HYBRID_LOWER
import matplotlib.pyplot as plt
from analyses.NHRC.nhrc_utils.analysis import ACCURACY_COLUMN, AUROC_COLUMN, MODEL_COLUMN, SCENARIO_COLUMN, SLEEP_ACCURACY_COLUMN, WASA_COLUMN
from analyses.NHRC.nhrc_utils.plotting import tri_plot_metrics
from analyses.NHRC.nhrc_utils.model_definitions import EXTRA_LAYERS_NAME, EXTRA_LOWER, LR_LOWER, NAIVE_LOWER
from collections import defaultdict
from analyses.NHRC.nhrc_utils.analysis import compute_evaluations_df


import pandas as pd

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'  # Set this before importing TensorFlow
import tensorflow as tf
tf.get_logger().setLevel('ERROR')

from analyses.NHRC.nhrc_utils.analysis import DF_COLUMNS, WASA_SLEEP_ACCURACY, auroc_balaccuracy_wasa

acc_hz_string = "dyn"
WASA_SLEEP_PERCENT = 93
STATIONARY_SET_NAME = "Stationary Data"
HYBRID_SET_NAME = "Hybrid Data"
SECOND_LINE = f"Sleep Accuracy ≈ {WASA_SLEEP_PERCENT}%"
SECOND_TITLE_LINE = f"Comparison of Stationary and Hybrid Data Metrics, {SECOND_LINE}"
TRIPLOT_DPI = 300

scenarios = SCENARIOS

In [ ]:
from collections import defaultdict
from analyses.NHRC.nhrc_utils.analysis import prepare_data

sets_prepro_data = defaultdict(dict)


for set_name in scenarios:
    print("Loading", set_name)
    sets_prepro_data[set_name] = np.load(f'./pre_processed_data/{set_name}/{set_name}_preprocessed_data_{acc_hz_string}.npy', allow_pickle=True).item()

stationary_data_bundle = prepare_data(sets_prepro_data['stationary'])
hybrid_data_bundle = prepare_data(sets_prepro_data['hybrid'])

In [ ]:


# rev_sets = reverse_dicts(rev_sets_prepro)
walch_keys = list(sets_prepro_data['stationary'].keys())

cnn_predictors = [
    tf.keras.models.load_model(f"/Users/eric/Engineering/Work/pisces/analyses/NHRC/evaluations/models/finetuning_{i}.keras")
    for i in walch_keys 
]

lr_predictors = [
    tf.keras.models.load_model(f"/Users/eric/Engineering/Work/pisces/analyses/NHRC/evaluations/models/lr_{i}.keras")
    for i in walch_keys 
]


In [ ]:



evaluations_df = compute_evaluations_df(
    walch_keys,
    stationary_data_bundle,
    hybrid_data_bundle,
    lr_predictors,
    cnn_predictors
)

fig, axs = plt.subplots(2, 3, figsize=(15, 10))

cnn_evaluations_df = evaluations_df[evaluations_df[MODEL_COLUMN] == EXTRA_LOWER]
cnn_evaluations_df = cnn_evaluations_df[cnn_evaluations_df[SLEEP_ACCURACY_COLUMN] == WASA_SLEEP_PERCENT]
cnn_stationary = cnn_evaluations_df[cnn_evaluations_df[SCENARIO_COLUMN] == STATIONARY_LOWER]
cnn_hybrid = cnn_evaluations_df[cnn_evaluations_df[SCENARIO_COLUMN] == HYBRID_LOWER]

for ax in axs.flatten():
    ax.set_ylim(0, 20)

tri_plot_metrics(evaluations_df=cnn_stationary, axs=axs[0], axs_set_name=STATIONARY_SET_NAME,
                 wasa_column=WASA_COLUMN, accuracy_column=ACCURACY_COLUMN, auroc_column=AUROC_COLUMN)
tri_plot_metrics(cnn_hybrid, axs=axs[1], axs_set_name=HYBRID_SET_NAME,
                 wasa_column=WASA_COLUMN, accuracy_column=ACCURACY_COLUMN, auroc_column=AUROC_COLUMN)


fig.suptitle(f"Mads Olsen's UNet + Arcascope {EXTRA_LAYERS_NAME} CNN\n{SECOND_TITLE_LINE}\nPISCES {acc_hz_string} Hz", fontsize=20)

fig.tight_layout()
fig.savefig(DEFAULT_EVALUATION_DIR.joinpath(f"{EXTRA_LOWER}_triplot_{WASA_SLEEP_PERCENT}_{acc_hz_string}.png"), dpi=TRIPLOT_DPI, bbox_inches='tight')

In [ ]:
acc_hz_string = "50"
sets_prepro_data = defaultdict(dict)


for set_name in scenarios:
    print("Loading", set_name)
    sets_prepro_data[set_name] = np.load(f'./pre_processed_data/{set_name}/{set_name}_preprocessed_data_{acc_hz_string}.npy', allow_pickle=True).item()

stationary_data_bundle_50 = prepare_data(sets_prepro_data['stationary'])
hybrid_data_bundle_50 = prepare_data(sets_prepro_data['hybrid'])

evaluations_df_50 = compute_evaluations_df(
    walch_keys,
    stationary_data_bundle_50,
    hybrid_data_bundle_50,
    lr_predictors,
    cnn_predictors
)

In [ ]:

fig, axs = plt.subplots(2, 3, figsize=(15, 10))

cnn_evaluations_df = evaluations_df_50[evaluations_df_50[MODEL_COLUMN] == EXTRA_LOWER]
cnn_evaluations_df = cnn_evaluations_df[cnn_evaluations_df[SLEEP_ACCURACY_COLUMN] == WASA_SLEEP_PERCENT]
cnn_stationary = cnn_evaluations_df[cnn_evaluations_df[SCENARIO_COLUMN] == STATIONARY_LOWER]
cnn_hybrid = cnn_evaluations_df[cnn_evaluations_df[SCENARIO_COLUMN] == HYBRID_LOWER]

for ax in axs.flatten():
    ax.set_ylim(0, 20)

tri_plot_metrics(evaluations_df=cnn_stationary, axs=axs[0], axs_set_name=STATIONARY_SET_NAME,
                 wasa_column=WASA_COLUMN, accuracy_column=ACCURACY_COLUMN, auroc_column=AUROC_COLUMN)
tri_plot_metrics(cnn_hybrid, axs=axs[1], axs_set_name=HYBRID_SET_NAME,
                 wasa_column=WASA_COLUMN, accuracy_column=ACCURACY_COLUMN, auroc_column=AUROC_COLUMN)


fig.suptitle(f"Mads Olsen's UNet + Arcascope {EXTRA_LAYERS_NAME} CNN\n{SECOND_TITLE_LINE}\nPISCES {acc_hz_string} Hz", fontsize=20)

fig.tight_layout()
fig.savefig(DEFAULT_EVALUATION_DIR.joinpath(f"{EXTRA_LOWER}_triplot_{WASA_SLEEP_PERCENT}.png"), dpi=TRIPLOT_DPI, bbox_inches='tight')

In [ ]:
from collections import defaultdict
from pathlib import Path

import matplotlib.pyplot as plt

from analyses.NHRC.nhrc_utils.analysis import SCENARIOS


specgram_folder = Path('/Users/eric/Engineering/Work/pisces/data_sets/specgrams')

specgram_data = defaultdict(dict)

scenarios = SCENARIOS
print(scenarios)

for file in specgram_folder.joinpath('NSR').glob('*.npy'):
    scenario, subject_id = file.stem.split('_')[:2]
    specgram_data[scenario][subject_id] = np.squeeze(np.load(file, allow_pickle=True))

In [ ]:
specgram_data.keys()

In [ ]:
keys = sorted(list(specgram_data['stationary'].keys()))

In [ ]:
print(keys)

In [ ]:
nsr_stationary = np.array([specgram_data['stationary'][subject_id] for subject_id in keys])
nsr_hybrid = np.array([specgram_data['hybrid'][subject_id] for subject_id in keys])

In [ ]:
fig, ax = plt.subplots(2, 2, figsize=(15, 10))
datas = [
    [stationary_data_bundle.spectrogram[0], hybrid_data_bundle.spectrogram[0]],
    [nsr_stationary[0], nsr_hybrid[0]]
]


for row in range(2):
    for col in range(2):
        array = np.squeeze(datas[row][col])
        ax[row][col].imshow(array.T, aspect='auto', vmin=-15, vmax=20)
        ax[0][col].set_title(['Stationary', 'Hybrid'][col])
    ax[row][0].set_ylabel(['Pisces', 'NSR'][row])

# Feed NSR exports into Pisces' MO evaluation loop

## Move NSR exports into bundles

In [ ]:
missed_keys = 0
rev_sets_prepro = sets_prepro_data
for key in rev_sets_prepro.keys():
    pisces_prepro = rev_sets_prepro[key]
    nsr_prepro = specgram_data.get(key)

    if nsr_prepro is None:
        print("No NSR data for", key)
        continue

    print("Processing", key)

    for iden in pisces_prepro.keys():
        diff = rev_sets_prepro[key][iden]['spectrogram'] - nsr_prepro[iden]
        # print(f"Diff: [{diff.min()}, {diff.max()}], μ: {diff.mean()}")
        if iden not in nsr_prepro.keys():
            missed_keys += 1
            print("Missing", iden, "from NSR", key)
            continue
        rev_sets_prepro[key][iden]['spectrogram.backup'] = rev_sets_prepro[key][iden]['spectrogram']
        rev_sets_prepro[key][iden]['spectrogram'] = nsr_prepro[iden]
diff = rev_sets_prepro['stationary'][iden]['spectrogram'] - rev_sets_prepro['hybrid'][iden]['spectrogram']
print(f"{iden} Diff: [{diff.min()}, {diff.max()}], μ: {diff.mean()}")
    


## Evaluations

In [ ]:


# rev_sets = reverse_dicts(rev_sets_prepro)
rev_sets = sets_prepro_data

hybrid_preprocessed_data = rev_sets['hybrid']
walch_preprocessed_data = rev_sets['stationary']
hybrid_data_bundle_NSR = prepare_data(hybrid_preprocessed_data)
stationary_data_bundle_NSR = prepare_data(walch_preprocessed_data)

evaluations_df_NSR = compute_evaluations_df(
    walch_keys,
    stationary_data_bundle_NSR,
    hybrid_data_bundle_NSR,
    lr_predictors,
    cnn_predictors
)

In [ ]:
from pisces.data_sets import DataSetObject

data_location = Path('/Users/eric/Engineering/Work/pisces/data_sets')

found_sets = DataSetObject.find_data_sets(data_location)
stationary_base = found_sets['walch_et_al']
hybrid_base = found_sets['hybrid_motion']

stationary_base.parse_data_sets()
hybrid_base.parse_data_sets()

In [ ]:
s0_id = stationary_base.ids[0]

s0_acc = stationary_base.get_feature_data('accelerometer', s0_id)
h0_acc = hybrid_base.get_feature_data('accelerometer', s0_id)

In [ ]:
plt.plot(h0_acc[:, 0], h0_acc[:, 1:], 'o')
plt.plot(s0_acc[:, 0], s0_acc[:, 1:], 'o')

In [ ]:
fig, ax = plt.subplots(2, 1, sharex=True)
ax[0].imshow(stationary_data_bundle.spectrogram[0].numpy().T, aspect='auto')
ax[1].imshow(hybrid_data_bundle.spectrogram[0].numpy().T, aspect='auto')

In [ ]:
(stationary_data_bundle.mo_predictions[0].numpy().T - hybrid_data_bundle.mo_predictions[0].numpy().T).std()

# It's the spectrograms
Specifically, the accelerometer resampling between NSR and pisces seems to be introducing differences! The Pisces-era eval loop below will discover the same outlier behavior as we saw in NSR.

In [ ]:

fig, axs = plt.subplots(2, 3, figsize=(15, 10))

cnn_evaluations_df = evaluations_df_NSR[evaluations_df_NSR[MODEL_COLUMN] == EXTRA_LOWER]
cnn_evaluations_df = cnn_evaluations_df[cnn_evaluations_df[SLEEP_ACCURACY_COLUMN] == WASA_SLEEP_PERCENT]
cnn_stationary = cnn_evaluations_df[cnn_evaluations_df[SCENARIO_COLUMN] == STATIONARY_LOWER]
cnn_hybrid = cnn_evaluations_df[cnn_evaluations_df[SCENARIO_COLUMN] == HYBRID_LOWER]

for ax in axs.flatten():
    ax.set_ylim(0, 20)

tri_plot_metrics(evaluations_df=cnn_stationary, axs=axs[0], axs_set_name=STATIONARY_SET_NAME,
                 wasa_column=WASA_COLUMN, accuracy_column=ACCURACY_COLUMN, auroc_column=AUROC_COLUMN)
tri_plot_metrics(cnn_hybrid, axs=axs[1], axs_set_name=HYBRID_SET_NAME,
                 wasa_column=WASA_COLUMN, accuracy_column=ACCURACY_COLUMN, auroc_column=AUROC_COLUMN)


fig.suptitle(f"Mads Olsen's UNet + Arcascope {EXTRA_LAYERS_NAME} CNN\n{SECOND_TITLE_LINE}\nNavySleepResearch", fontsize=20)

fig.tight_layout()
fig.savefig(DEFAULT_EVALUATION_DIR.joinpath(f"{EXTRA_LOWER}_triplot_{WASA_SLEEP_PERCENT}.png"), dpi=TRIPLOT_DPI, bbox_inches='tight')

In [ ]:


fig, axs = plt.subplots(2, 3, figsize=(15, 10))

naive_evaluations_df = evaluations_df_NSR[evaluations_df_NSR[MODEL_COLUMN] == NAIVE_LOWER]
naive_evaluations_df = naive_evaluations_df[naive_evaluations_df[SLEEP_ACCURACY_COLUMN] == WASA_SLEEP_PERCENT]
naive_stationary = naive_evaluations_df[naive_evaluations_df[SCENARIO_COLUMN] == STATIONARY_LOWER]
naive_hybrid = naive_evaluations_df[naive_evaluations_df[SCENARIO_COLUMN] == HYBRID_LOWER]

tri_plot_metrics(naive_stationary, axs=axs[0], axs_set_name=STATIONARY_SET_NAME, 
                 wasa_column=WASA_COLUMN, accuracy_column=ACCURACY_COLUMN, auroc_column=AUROC_COLUMN)
tri_plot_metrics(naive_hybrid, axs=axs[1], axs_set_name=HYBRID_SET_NAME, 
                 wasa_column=WASA_COLUMN, accuracy_column=ACCURACY_COLUMN, auroc_column=AUROC_COLUMN)
for ax in axs.flatten():
    ax.set_ylim(0, 20)

fig.suptitle(f"Mads Olsen's UNet => Naive 1 - Pr(W)\n{SECOND_TITLE_LINE}", fontsize=20)

fig.tight_layout()
fig.savefig(DEFAULT_EVALUATION_DIR.joinpath(f"{NAIVE_LOWER}_triplot_{WASA_SLEEP_PERCENT}.png"), dpi=TRIPLOT_DPI, bbox_inches='tight')

## Tri plot for logistic regression CNN

In [ ]:
fig, axs = plt.subplots(2, 3, figsize=(15, 10))

lr_evaluations_df = evaluations_df[evaluations_df[MODEL_COLUMN] == LR_LOWER]
lr_evaluations_df = lr_evaluations_df[lr_evaluations_df[SLEEP_ACCURACY_COLUMN] == WASA_SLEEP_PERCENT]
lr_stationary = lr_evaluations_df[lr_evaluations_df[SCENARIO_COLUMN] == STATIONARY_LOWER]
lr_hybrid = lr_evaluations_df[lr_evaluations_df[SCENARIO_COLUMN] == HYBRID_LOWER]

tri_plot_metrics(lr_stationary, axs=axs[0], axs_set_name=STATIONARY_SET_NAME,
                 wasa_column=WASA_COLUMN, accuracy_column=ACCURACY_COLUMN, auroc_column=AUROC_COLUMN)
tri_plot_metrics(lr_hybrid, axs=axs[1], axs_set_name=HYBRID_SET_NAME,
                 wasa_column=WASA_COLUMN, accuracy_column=ACCURACY_COLUMN, auroc_column=AUROC_COLUMN)

for ax in axs.flatten():
    ax.set_ylim(0, 20)

fig.suptitle(f"Logistic Regression in TF\n{SECOND_TITLE_LINE}", fontsize=20)

fig.tight_layout()
fig.savefig(DEFAULT_EVALUATION_DIR.joinpath(f"{LR_LOWER}_triplot_{WASA_SLEEP_PERCENT}.png"), dpi=TRIPLOT_DPI, bbox_inches='tight')